In [1]:
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
from scipy.stats import zscore, norm

### Teams

In [2]:
# File from Kaggle, all possible team spellings (to get to the TeamID)
spellings = pd.read_csv('ncaam-march-mania-2021/MTeamSpellings.csv', encoding = 'ISO-8859-1')
spellings.head()

,TeamNameSpelling,TeamID
0,a&m-corpus chris,1394
1,a&m-corpus christi,1394
2,abilene chr,1101
3,abilene christian,1101
4,abilene-christian,1101


In [3]:
# Put the team names in the same format (lowercase no punctuation) for joins
spellings['TeamNameSpelling'] = spellings['TeamNameSpelling'].str.replace('[^a-zA-Z&.()\' ]+',' ').str.lower()
spellings['TeamNameSpelling'] = spellings['TeamNameSpelling'].str.replace('[^a-z&.()\' ]+','')

### Helper Functions and Variables

In [4]:
# last days before tournament starts, these are the days we want to scrape to avoid any leakage
last_days = {2008: '2008-03-19',
             2009: '2009-03-18',
             2010: '2010-03-17',
            2011: '2011-03-16',
            2012: '2012-03-14',
            2013: '2013-03-20',
            2014: '2014-03-19',
            2015: '2015-03-18',
            2016: '2016-03-16',
            2017: '2017-03-15',
            2018: '2018-03-14',
            2019: '2019-03-20'}

# seasons to scrape, 2008-2019
seasons = [2008 + i for i in range(12)]

In [5]:
nans = lambda df: df[df.isnull().any(axis=1)]  # Function to print out rows with null values

In [6]:
# returns name of team that is in the spellings csv
def fix_name(row):
    if row['Team'] == 'st marys':
        return 'st marys ca'
    elif row['Team'] == 'wins salem' or row['Team'] == 'winston salem st.':
        return 'winston salem'
    elif row['Team'] == 'w virginia':
        return 'west virginia'
    elif row['Team'] == 'n carolina':
        return 'north carolina'
    elif row['Team'] == 'tx christian':
        return 'tcu'
    elif row['Team'] == 'va tech':
        return 'virginia tech'
    elif row['Team'] == 'miss state':
        return 'mississippi st'
    elif row['Team'] == 'st bonavent':
        return 'st bonaventure'
    elif row['Team'] == 'loyola chi':
        return 'loyola chicago'
    elif row['Team'] == 's methodist':
        return 'smu'
    elif row['Team'] == 'n mex state':
        return 'new mexico st'
    elif row['Team'] == 's carolina':
        return 'south carolina'
    elif row['Team'] == 'boston col':
        return 'boston college'
    elif row['Team'] == 'e tenn st':
        return 'etsu'
    elif row['Team'] == 'nc grnsboro':
        return 'unc greensboro'
    elif row['Team'] == 'central fl':
        return 'ucf'
    elif row['Team'] == 'utah val st':
        return 'utah valley state'
    elif row['Team'] == 'northeastrn':
        return 'northeastern'
    elif row['Team'] == 'ga tech':
        return 'georgia tech'
    elif row['Team'] == 'col charlestn':
        return 'college of charleston'
    elif row['Team'] == 'st josephs':
        return 'st josephs pa'
    elif row['Team'] == 'u penn':
        return 'penn'
    elif row['Team'] == 'ste f austin':
        return 'stephen f austin'
    elif row['Team'] == 'fla gulf cst':
        return 'florida gulf coast'
    elif row['Team'] == 'grd canyon':
        return 'grand canyon'
    elif row['Team'] == 'tx arlington':
        return 'ut arlington'
    elif row['Team'] == 'n iowa':
        return 'northern iowa'
    elif row['Team'] == 'la tech':
        return 'louisiana tech'
    elif row['Team'] == 'wm & mary':
        return 'william & mary'
    elif row['Team'] == 'jksnville st':
        return 'jacksonville st'
    elif row['Team'] == 'app state':
        return 'appalachian st'
    elif row['Team'] == 'san fransco':
        return 'san francisco'
    elif row['Team'] == 'e washingtn':
        return 'eastern washington'
    elif row['Team'] == 'geo wshgtn':
        return 'george washington'
    elif row['Team'] == 'u mass':
        return 'umass'
    elif row['Team'] == 'maryland bc':
        return 'umbc'
    elif row['Team'] == 'wash state':
        return 'washington st'
    elif row['Team'] == 'tx san ant':
        return 'utsa'
    elif row['Team'] == 'st fran (pa)' or row['Team'] == 'st. francis pa':
        return 'st francis pa'
    elif row['Team'] == 'miami oh':
        return 'miami ohio'
    elif row['Team'] == 'geo mason':
        return 'george mason'
    elif row['Team'] == 'wi milwkee':
        return 'milwaukee'
    elif row['Team'] == 'tn state':
        return 'tennessee st'
    elif row['Team'] == 'tn tech':
        return 'tennessee tech'
    elif row['Team'] == 'nc wilmgton':
        return 'unc wilmington'
    elif row['Team'] == 's alabama':
        return 'south alabama'
    elif row['Team'] == 'lg beach st':
        return 'long beach st'
    elif row['Team'] == 'james mad':
        return 'james madison'
    elif row['Team'] == 'sam hous st':
        return 'sam houston st'
    elif row['Team'] == 'cs bakersfld' or row['Team'] == 'cal st. bakersfield':
        return 'cal state bakersfield'
    elif row['Team'] == 'loyola mymt':
        return 'loyola marymount'
    elif row['Team'] == 's mississippi':
        return 'southern miss'
    elif row['Team'] == 'bowling grn':
        return 'bowling green'
    elif row['Team'] == 'tx el paso':
        return 'utep'
    elif row['Team'] == 'n hampshire':
        return 'new hampshire'
    elif row['Team'] == 'rob morris':
        return 'robert morris'
    elif row['Team'] == 'wi grn bay':
        return 'green bay'
    elif row['Team'] == 'charl south':
        return 'charleston southern'
    elif row['Team'] == 'abl christian':
        return 'abilene christian'
    elif row['Team'] == 'gard webb':
        return 'gardner webb'
    elif row['Team'] == 'tx pan am':
        return 'texas pan american'
    elif row['Team'] == 'se missouri' or row['Team'] == 'southeast missouri st.':
        return 'se missouri st'
    elif row['Team'] == 'neb omaha':
        return 'omaha'
    elif row['Team'] == 's florida':
        return 'south florida'
    elif row['Team'] == 'mass lowell':
        return 'umass lowell'
    elif row['Team'] == 'e carolina':
        return 'east carolina'
    elif row['Team'] == 'tx a&m cc' or row['Team'] == 'texas a&m corpus chris':
        return 'a&m corpus chris'
    elif row['Team'] == 's utah':
        return 'southern utah'
    elif row['Team'] == 'n florida':
        return 'north florida'
    elif row['Team'] == 'sacred hrt':
        return 'sacred heart'
    elif row['Team'] == 'st fran (ny)':
        return 'st francis ny'
    elif row['Team'] == 'ar lit rock':
        return 'arkansas little rock'
    elif row['Team'] == 'beth cook':
        return 'bethune cookman'
    elif row['Team'] == 'sac state':
        return 'sacramento st'
    elif row['Team'] == 'siu edward':
        return 'southern illinois'
    elif row['Team'] == 'youngs st':
        return 'youngstown st'
    elif row['Team'] == 'nw state':
        return 'northwestern st'
    elif row['Team'] == 'cal st nrdge':
        return 'cal state northridge'
    elif row['Team'] == 'ark pine bl':
        return 'arkansas pine bluff'
    elif row['Team'] == 'va military':
        return 'vmi'
    elif row['Team'] == 'incar word':
        return 'incarnate word'
    elif row['Team'] == 'n arizona':
        return 'northern arizona' 
    elif row['Team'] == 's car state':
        return 'south carolina state'
    elif row['Team'] == 'nw st':
        return 'northwestern st'
    elif row['Team'] == 'miss val st' or row['Team'] == 'mississippi valley st.':
        return 'mississippi valley state'
    elif row['Team'] == 'maryland es':
        return 'umes'
    elif row['Team'] == 'alab a&m':
        return 'alabama a&m' 
    elif row['Team'] == 'n alabama':
        return 'north alabama'
    elif row['Team'] == 'la lafayette':
        return 'louisiana lafayette'
    elif row['Team'] == 'grambling st':
        return 'grambling state'
    elif row['Team'] == 'ut rio grande valley':
        return 'texas rio grande valley'
    elif row['Team'] == 'liu brooklyn ( )':
        return 'liu brooklyn'
    else:
        return row['Team']

### Teamrank Ratings

Scraping teamrankings.com for their CBB Ratings

In [7]:
# scrape each season's ratings
season_list = []
team_list = []
rating_list = []
for season in seasons:
    time.sleep(5)
    teamrank_url = 'https://www.teamrankings.com/ncaa-basketball/ranking/predictive-by-other?date=' + last_days[season]
    teamrank_page = requests.get(teamrank_url)
    teamrank_soup = BeautifulSoup(teamrank_page.content, 'lxml')
    teamrank_rows = teamrank_soup.select('tbody tr')
    for row in teamrank_rows:
        anchor = row.select('.nowrap')[0].select('a')
        if not anchor and not row.select('.nowrap')[0].get_text().lower().startswith('liu'): # this if statement is neccessary due to teamrank
            continue
        season_list.append(season)
        if anchor:
            team_list.append(anchor[0].get_text())
        else:
            team_list.append(row.select('.nowrap')[0].get_text())
        rating_list.append(row.find_all('td')[2].get_text())  # magic number
teamrank_temp = pd.DataFrame({'Season': season_list, 'Team': team_list, 'TeamrankRating': rating_list})

In [8]:
teamrank = teamrank_temp.copy()
teamrank.head()

,Season,Team,TeamrankRating
0,2008,Kansas,32.4
1,2008,N Carolina,29.4
2,2008,Memphis,28.7
3,2008,Duke,28.5
4,2008,UCLA,28.2


In [9]:
teamrank.tail()

,Season,Team,TeamrankRating
4170,2019,Alcorn State,-17.7
4171,2019,Miss Val St,-18.3
4172,2019,Maryland ES,-19.7
4173,2019,Delaware St,-21.6
4174,2019,Chicago St,-21.7


In [10]:
# Put the team names in the same format (lowercase no punctuation) for joins
teamrank['Team'] = teamrank['Team'].str.replace('[^a-zA-Z&.()\' ]+',' ').str.lower()
teamrank['Team'] = teamrank['Team'].str.replace('[^a-z&.()\' ]+','')

In [11]:
# fix the names in order to join to get the team id
teamrank['Team'] = teamrank.apply(fix_name, axis = 1)

In [12]:
teamrank_teams = pd.merge(teamrank, spellings, how = 'left', left_on = 'Team', right_on = 'TeamNameSpelling')
nans(teamrank_teams)

,Season,Team,TeamrankRating,TeamNameSpelling,TeamID
406,2008,dixie state,-3.3,NaN,NaN


The above teams were not in division 1

In [13]:
teamrank_teams = teamrank_teams[['TeamID', 'Season', 'TeamrankRating']].drop_duplicates()
teamrank_teams.head()

,TeamID,Season,TeamrankRating
0,1242.0,2008,32.4
1,1314.0,2008,29.4
3,1272.0,2008,28.7
4,1181.0,2008,28.5
5,1417.0,2008,28.2


### Trank Ratings

Getting Trank ratings from barttorvik.com

In [14]:
trank = pd.read_csv('http://barttorvik.com/teamslicejson.php?year=2008&csv=1&type=R', header = None)
trank = trank[[0, 1, 2, 3, 26]]
trank.columns = ['Team', 'OE', 'DE', 'TrankRating', 'Tempo']
trank['Season'] = 2008
for season in seasons:
    time.sleep(3)
    if season != 2008:
        trank_url = 'http://barttorvik.com/teamslicejson.php?year=' + str(season) + '&csv=1&type=R'
        trank_temp = pd.read_csv(trank_url, header = None)
        trank_temp = trank_temp[[0, 1, 2, 3, 26]]
        trank_temp.columns = ['Team', 'OE', 'DE', 'TrankRating', 'Tempo']
        trank_temp['Season'] = season
        trank = pd.concat([trank, trank_temp])
trank = trank.reset_index(drop = True)
trank.head()

,Team,OE,DE,TrankRating,Tempo,Season
0,Jackson St.,91.259939,107.681226,0.129789,72.4,2008
1,Mississippi,112.217890,98.272341,0.821427,71.1,2008
2,TCU,98.968705,99.941178,0.471918,69.0,2008
3,Albany,101.159577,103.145314,0.444343,65.3,2008
4,Wyoming,99.388001,102.575619,0.410223,70.5,2008


In [15]:
trank.tail()

,Team,OE,DE,TrankRating,Tempo,Season
4172,Fordham,96.800091,102.817876,0.333237,66.2,2019
4173,Alabama St.,93.767566,110.787449,0.128070,68.4,2019
4174,Wichita St.,105.897571,96.538612,0.743471,69.2,2019
4175,Oakland,107.713950,108.870026,0.469346,69.7,2019
4176,Boise St.,107.634044,102.400989,0.639494,66.9,2019


In [16]:
# Calculate season average tempo
avg_tempo = trank.groupby('Season').agg(AvgTempo = ('Tempo', 'mean'))

# join to trank
trank = pd.merge(trank, avg_tempo, on = 'Season')

trank.head()

,Team,OE,DE,TrankRating,Tempo,Season,AvgTempo
0,Jackson St.,91.259939,107.681226,0.129789,72.4,2008,67.317889
1,Mississippi,112.217890,98.272341,0.821427,71.1,2008,67.317889
2,TCU,98.968705,99.941178,0.471918,69.0,2008,67.317889
3,Albany,101.159577,103.145314,0.444343,65.3,2008,67.317889
4,Wyoming,99.388001,102.575619,0.410223,70.5,2008,67.317889


In [17]:
# Put the team names in the same format (lowercase no punctuation) for joins
trank['Team'] = trank['Team'].str.replace('[^a-zA-Z&.()\' ]+',' ').str.lower()
trank['Team'] = trank['Team'].str.replace('[^a-z&.()\' ]+','')

In [18]:
# fix the names in order to join to get the team id
trank['Team'] = trank.apply(fix_name, axis = 1)

In [19]:
trank_copy = trank.copy()
trank_teams = pd.merge(trank_copy, spellings, how = 'left', left_on = 'Team', right_on = 'TeamNameSpelling')
nans(trank_teams)

,Team,OE,DE,TrankRating,Tempo,Season,AvgTempo,TeamNameSpelling,TeamID


In [20]:
trank_teams = trank_teams[['TeamID', 'Season', 'TrankRating', 'OE', 'DE', 'Tempo', 'AvgTempo']].drop_duplicates()
trank_teams.head()

,TeamID,Season,TrankRating,OE,DE,Tempo,AvgTempo
0,1238,2008,0.129789,91.259939,107.681226,72.4,67.317889
1,1279,2008,0.821427,112.217890,98.272341,71.1,67.317889
2,1395,2008,0.471918,98.968705,99.941178,69.0,67.317889
3,1107,2008,0.444343,101.159577,103.145314,65.3,67.317889
4,1461,2008,0.410223,99.388001,102.575619,70.5,67.317889


In [21]:
teams = pd.merge(teamrank_teams, trank_teams, how = 'inner', on = ['Season', 'TeamID'])
teams.head()

,TeamID,Season,TeamrankRating,TrankRating,OE,DE,Tempo,AvgTempo
0,1242.0,2008,32.4,0.981585,120.970641,85.610492,69.5,67.317889
1,1314.0,2008,29.4,0.957196,120.240748,91.770372,75.1,67.317889
2,1272.0,2008,28.7,0.969683,113.254181,83.789458,70.7,67.317889
3,1181.0,2008,28.5,0.960742,117.213494,88.761128,73.7,67.317889
4,1417.0,2008,28.2,0.966422,116.350781,86.873950,66.2,67.317889


In [22]:
# set ratings as floats for calculations
teams['TeamrankRating'] = teams['TeamrankRating'].astype(float)
teams['TrankRating'] = teams['TrankRating'].astype(float)

# Calculate Z score of ratings to put them on same scale
teams['TeamrankZScore'] = teams.groupby('Season')['TeamrankRating'].transform(lambda x: zscore(x))
teams['TrankZScore'] = teams.groupby('Season')['TrankRating'].transform(lambda x: zscore(x))
teams.head()

,TeamID,Season,TeamrankRating,TrankRating,OE,DE,Tempo,AvgTempo,TeamrankZScore,TrankZScore
0,1242.0,2008,32.4,0.981585,120.970641,85.610492,69.5,67.317889,2.825912,1.819909
1,1314.0,2008,29.4,0.957196,120.240748,91.770372,75.1,67.317889,2.488078,1.728525
2,1272.0,2008,28.7,0.969683,113.254181,83.789458,70.7,67.317889,2.409250,1.775314
3,1181.0,2008,28.5,0.960742,117.213494,88.761128,73.7,67.317889,2.386728,1.741812
4,1417.0,2008,28.2,0.966422,116.350781,86.873950,66.2,67.317889,2.352944,1.763096


In [23]:
# Take a weighted average of ratings
teams['WeightedRating'] = 0.55 * teams['TrankZScore'] + 0.45 * teams['TeamrankZScore']
teams = teams.drop(columns = ['TrankZScore', 'TeamrankZScore'])
teams.head()

,TeamID,Season,TeamrankRating,TrankRating,OE,DE,Tempo,AvgTempo,WeightedRating
0,1242.0,2008,32.4,0.981585,120.970641,85.610492,69.5,67.317889,2.272611
1,1314.0,2008,29.4,0.957196,120.240748,91.770372,75.1,67.317889,2.070324
2,1272.0,2008,28.7,0.969683,113.254181,83.789458,70.7,67.317889,2.060585
3,1181.0,2008,28.5,0.960742,117.213494,88.761128,73.7,67.317889,2.032024
4,1417.0,2008,28.2,0.966422,116.350781,86.873950,66.2,67.317889,2.028528


### Tournament Seeds

In [24]:
seeds = pd.read_csv('ncaam-march-mania-2021/MNCAATourneySeeds.csv')
seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [25]:
# merge seeds with team data
teams = pd.merge(teams, seeds, on = ['Season', 'TeamID'], how = 'inner')
teams.head()

,TeamID,Season,TeamrankRating,TrankRating,OE,DE,Tempo,AvgTempo,WeightedRating,Seed
0,1242.0,2008,32.4,0.981585,120.970641,85.610492,69.5,67.317889,2.272611,X01
1,1314.0,2008,29.4,0.957196,120.240748,91.770372,75.1,67.317889,2.070324,W01
2,1272.0,2008,28.7,0.969683,113.254181,83.789458,70.7,67.317889,2.060585,Y01
3,1181.0,2008,28.5,0.960742,117.213494,88.761128,73.7,67.317889,2.032024,Z02
4,1417.0,2008,28.2,0.966422,116.350781,86.873950,66.2,67.317889,2.028528,Z01


In [26]:
# extract just the seed number, no need for region here
def clean_seeds(row):
    return int(row['Seed'][1:3])

teams['Seed'] = teams.apply(clean_seeds, axis = 1)
teams.head()

,TeamID,Season,TeamrankRating,TrankRating,OE,DE,Tempo,AvgTempo,WeightedRating,Seed
0,1242.0,2008,32.4,0.981585,120.970641,85.610492,69.5,67.317889,2.272611,1
1,1314.0,2008,29.4,0.957196,120.240748,91.770372,75.1,67.317889,2.070324,1
2,1272.0,2008,28.7,0.969683,113.254181,83.789458,70.7,67.317889,2.060585,1
3,1181.0,2008,28.5,0.960742,117.213494,88.761128,73.7,67.317889,2.032024,2
4,1417.0,2008,28.2,0.966422,116.350781,86.873950,66.2,67.317889,2.028528,1


### Regular Season Stats

Kaggle provides a csv file of the regular season results for all torunament teams since 2003. I will use this dataset to get the remaining offensive and defensive stats I need for my analysis.

In [27]:
reg_season = pd.read_csv('ncaam-march-mania-2021/MRegularSeasonDetailedResults.csv')
reg_season.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [28]:
reg_season = reg_season.query('Season > 2007')  # because these are the years we have efficiency data

In [29]:
# Score differential per posseassion
reg_season['ScoreDiffPerPoss'] = 2 * (reg_season['WScore'] - reg_season['LScore']) / (reg_season['WFGA'] + reg_season['WTO'] + 0.44 * reg_season['WFTA'] - reg_season['WOR'] + reg_season['LFGA'] + reg_season['LTO'] + 0.44 * reg_season['LFTA'] - reg_season['LOR'])

# Adjust score differential for home court
def adj_score_for_location(row):
    if row['WLoc'] == 'H':
        return row['ScoreDiffPerPoss'] - 0.05
    elif row['WLoc'] == 'A':
        return row['ScoreDiffPerPoss'] + 0.05
    else:
        return row['ScoreDiffPerPoss']
    
reg_season['AdjScoreDiffPerPoss'] = reg_season.apply(adj_score_for_location, axis = 1)

In [30]:
# Get trank offensive efficiency and defensive efficiency
trank_ratings = trank_teams[['Season', 'TeamID', 'OE', 'DE']]

# Get stats I need from regrular season stats
my_data = reg_season[['Season', 'WTeamID', 'LTeamID', 'WFGM', 'LFGM', 'WFGA', 'LFGA', 'WFGM3', 'LFGM3', 'WFGA3', 'LFGA3', 'WFTM', 'LFTM', 'WFTA', 'LFTA', 'WAst', 'LAst', 'WTO', 'LTO', 'WOR', 'LOR', 'WDR', 'LDR', 'AdjScoreDiffPerPoss']]

# join stats and trank ratings for winning team
my_data = pd.merge(my_data, trank_ratings, left_on = ['Season', 'WTeamID'], right_on = ['Season', 'TeamID']).rename(columns = {'OE': 'WOE', 'DE': 'WDE'})

# join stats and trank ratings for losing team
my_data = pd.merge(my_data, trank_ratings, how = 'outer', left_on = ['Season', 'LTeamID'], right_on = ['Season', 'TeamID']).rename(columns = {'OE': 'LOE', 'DE': 'LDE'})

my_data = my_data.drop(columns = ['TeamID_x', 'TeamID_y'])

In [31]:
# What Trank predicted the adjusted score differential per possession would be for each game
my_data['PredictedAdjScoreDiffPerPoss'] = (my_data['WOE'] + my_data['LDE'] - (my_data['WDE'] + my_data['LOE'])) / 100
my_data.head()

,Season,WTeamID,LTeamID,WFGM,LFGM,WFGA,LFGA,WFGM3,LFGM3,WFGA3,...,WOR,LOR,WDR,LDR,AdjScoreDiffPerPoss,WOE,WDE,LOE,LDE,PredictedAdjScoreDiffPerPoss
0,2008,1272.0,1404.0,37.0,24.0,83.0,77.0,11.0,10.0,35.0,...,21.0,20.0,37.0,27.0,0.313764,113.254181,83.789458,102.686157,111.847766,0.386263
1,2008,1146.0,1404.0,27.0,25.0,63.0,68.0,6.0,8.0,26.0,...,13.0,11.0,35.0,30.0,0.089134,91.277499,102.858215,102.686157,111.847766,-0.024191
2,2008,1424.0,1404.0,30.0,16.0,63.0,60.0,8.0,4.0,25.0,...,14.0,17.0,31.0,25.0,0.322386,108.238028,93.854572,102.686157,111.847766,0.235451
3,2008,1280.0,1404.0,33.0,25.0,60.0,78.0,10.0,11.0,22.0,...,12.0,19.0,38.0,19.0,0.151005,106.343003,90.455829,102.686157,111.847766,0.250488
4,2008,1293.0,1404.0,29.0,27.0,57.0,64.0,7.0,9.0,15.0,...,10.0,13.0,28.0,20.0,0.109916,101.883604,105.622235,102.686157,111.847766,0.054230


In [32]:
# w_data is data for games in which the team won, and l_data is data for the games in which the team lost
w_data = my_data.groupby(['Season', 'WTeamID']).sum().drop(columns = ['LTeamID', 'AdjScoreDiffPerPoss', 'PredictedAdjScoreDiffPerPoss', 'WOE', 'WDE', 'LOE', 'LDE']).reset_index()
l_data = my_data.groupby(['Season', 'LTeamID']).sum().drop(columns = ['WTeamID', 'AdjScoreDiffPerPoss', 'PredictedAdjScoreDiffPerPoss', 'WOE', 'WDE', 'LOE', 'LDE']).reset_index()
w_data.head()

,Season,WTeamID,WFGM,LFGM,WFGA,LFGA,WFGM3,LFGM3,WFGA3,LFGA3,...,WFTA,LFTA,WAst,LAst,WTO,LTO,WOR,LOR,WDR,LDR
0,2008,1102.0,285.0,244.0,587.0,672.0,114.0,97.0,274.0,295.0,...,286.0,203.0,197.0,144.0,172.0,211.0,69.0,151.0,315.0,281.0
1,2008,1103.0,577.0,493.0,1216.0,1165.0,208.0,116.0,487.0,374.0,...,486.0,412.0,328.0,252.0,275.0,375.0,229.0,238.0,489.0,476.0
2,2008,1104.0,504.0,413.0,1038.0,1026.0,135.0,125.0,316.0,384.0,...,389.0,319.0,293.0,204.0,218.0,238.0,242.0,201.0,463.0,378.0
3,2008,1105.0,244.0,224.0,616.0,617.0,70.0,49.0,197.0,175.0,...,316.0,238.0,113.0,119.0,169.0,228.0,139.0,155.0,287.0,277.0
4,2008,1106.0,459.0,362.0,947.0,934.0,111.0,89.0,274.0,272.0,...,354.0,340.0,265.0,173.0,211.0,194.0,198.0,168.0,469.0,342.0


In [33]:
l_data.head()

,Season,LTeamID,WFGM,LFGM,WFGA,LFGA,WFGM3,LFGM3,WFGA3,LFGA3,...,WFTA,LFTA,WAst,LAst,WTO,LTO,WOR,LOR,WDR,LDR
0,2008,1102.0,291.0,234.0,643.0,600.0,115.0,85.0,292.0,256.0,...,262.0,254.0,194.0,137.0,169.0,183.0,112.0,78.0,331.0,283.0
1,2008,1103.0,257.0,210.0,513.0,558.0,59.0,68.0,156.0,230.0,...,216.0,222.0,135.0,116.0,157.0,146.0,107.0,140.0,236.0,179.0
2,2008,1104.0,425.0,413.0,901.0,972.0,133.0,97.0,343.0,324.0,...,335.0,285.0,248.0,222.0,229.0,211.0,180.0,206.0,402.0,349.0
3,2008,1105.0,382.0,315.0,811.0,853.0,92.0,86.0,227.0,277.0,...,341.0,259.0,217.0,137.0,250.0,248.0,177.0,190.0,392.0,300.0
4,2008,1106.0,259.0,249.0,576.0,602.0,65.0,61.0,169.0,213.0,...,239.0,155.0,142.0,126.0,131.0,146.0,128.0,124.0,261.0,224.0


In [34]:
wl_data = pd.merge(w_data, l_data, left_on = ['Season', 'WTeamID'], right_on = ['Season', 'LTeamID'], how = 'outer')
wl_data = wl_data.fillna(0)
# must do a outer join and fill NaNs with zeros due to undefeated teams

In [35]:
# Caculate season stats
stats = pd.DataFrame()
stats['Season'] = wl_data['Season']
stats['TeamID'] = wl_data['WTeamID']
stats['3ptRate'] = (wl_data['WFGA3_x'] + wl_data['LFGA3_y']) / (wl_data['WFGA_x'] + wl_data['LFGA_y'])
stats['Ast%'] = (wl_data['WAst_x'] + wl_data['LAst_y']) / (wl_data['WFGM_x'] + wl_data['LFGM_y'])
stats['FT%'] = (wl_data['WFTM_x'] + wl_data['LFTM_y']) / (wl_data['WFTA_x'] + wl_data['LFTA_y'])
stats['OppFT%'] = (wl_data['WFTM_y'] + wl_data['LFTM_x']) / (wl_data['WFTA_y'] + wl_data['LFTA_x'])
stats['Opp3ptRate'] = (wl_data['WFGA3_y'] + wl_data['LFGA3_x']) / (wl_data['WFGA_y'] + wl_data['LFGA_x'])
stats['OppAst%'] = (wl_data['WAst_y'] + wl_data['LAst_x']) / (wl_data['WFGM_y'] + wl_data['LFGM_x'])
stats['EFG%'] = (wl_data['WFGM_x'] + wl_data['LFGM_y'] + .5 * wl_data['WFGM3_x'] + .5 * wl_data['LFGM3_y']) / (wl_data['WFGA_x'] + wl_data['LFGA_y'])
stats['EFGD%'] = (wl_data['WFGM_y'] + wl_data['LFGM_x'] + .5 * wl_data['WFGM3_y'] + .5 * wl_data['LFGM3_x']) / (wl_data['WFGA_y'] + wl_data['LFGA_x'])
stats['TOR%'] = (wl_data['WTO_x'] + wl_data['LTO_y']) / (wl_data['WFGA_x'] + wl_data['LFGA_y'] - wl_data['WOR_x'] - wl_data['LOR_y'] + wl_data['WTO_x'] + wl_data['LTO_y'] + .44 * (wl_data['WFTA_x'] + wl_data['LFTA_y']))
stats['TORD%'] = (wl_data['WTO_y'] + wl_data['LTO_x']) / (wl_data['WFGA_y'] + wl_data['LFGA_x'] - wl_data['WOR_y'] - wl_data['LOR_x'] + wl_data['WTO_y'] + wl_data['LTO_x'] + .44 * (wl_data['WFTA_y'] + wl_data['LFTA_x']))
stats['ORB%'] = (wl_data['WOR_x'] + wl_data['LOR_y']) / (wl_data['WOR_x'] + wl_data['LOR_y'] + wl_data['WDR_y'] + wl_data['LDR_x'])
stats['OppORB%'] = 1 - (wl_data['WDR_x'] + wl_data['LDR_y']) / (wl_data['WOR_y'] + wl_data['LOR_x'] + wl_data['WDR_x'] + wl_data['LDR_y'])
stats['FTR'] = (wl_data['WFTA_x'] + wl_data['LFTA_y']) / (wl_data['WFGA_x'] + wl_data['LFGA_y'])
stats['FTRD'] = (wl_data['WFTA_y'] + wl_data['LFTA_x']) / (wl_data['WFGA_y'] + wl_data['LFGA_x'])
stats['2P%'] = (wl_data['WFGM_x'] + wl_data['LFGM_y'] - (wl_data['WFGM3_x'] + wl_data['LFGM3_y'])) / (wl_data['WFGA_x'] + wl_data['LFGA_y'] - (wl_data['WFGA3_x'] + wl_data['LFGA3_y']))
stats['2P%D'] = (wl_data['WFGM_y'] + wl_data['LFGM_x'] - (wl_data['WFGM3_y'] + wl_data['LFGM3_x'])) / (wl_data['WFGA_y'] + wl_data['LFGA_x'] - (wl_data['WFGA3_y'] + wl_data['LFGA3_x']))
stats['3P%'] = (wl_data['WFGM3_x'] + wl_data['LFGM3_y']) / (wl_data['WFGA3_x'] + wl_data['LFGA3_y'])
stats['3P%D'] = (wl_data['WFGM3_y'] + wl_data['LFGM3_x']) / (wl_data['WFGA3_y'] + wl_data['LFGA3_x'])
stats.head()

,Season,TeamID,3ptRate,Ast%,FT%,OppFT%,Opp3ptRate,OppAst%,EFG%,EFGD%,TOR%,TORD%,ORB%,OppORB%,FTR,FTRD,2P%,2P%D,3P%,3P%D
0,2008,1102.0,0.446504,0.643545,0.694444,0.694624,0.446388,0.631776,0.521061,0.487452,0.217445,0.232189,0.193676,0.305459,0.454928,0.353612,0.487062,0.443681,0.375472,0.361158
1,2008,1103.0,0.404171,0.564168,0.707627,0.684713,0.315852,0.516000,0.521421,0.499106,0.196957,0.248445,0.341351,0.340573,0.399098,0.374255,0.483444,0.500871,0.384937,0.330189
2,2008,1104.0,0.318408,0.561614,0.620178,0.718654,0.377270,0.539379,0.513930,0.501816,0.187536,0.202976,0.364821,0.319363,0.335323,0.339388,0.500000,0.483333,0.362500,0.354883
3,2008,1105.0,0.322668,0.447227,0.686957,0.697755,0.281513,0.554455,0.433628,0.473739,0.230387,0.261379,0.329659,0.361262,0.391423,0.405462,0.405025,0.453216,0.329114,0.350746
4,2008,1106.0,0.314396,0.552260,0.646365,0.640760,0.292053,0.507246,0.512589,0.462252,0.197460,0.181184,0.348108,0.299292,0.328599,0.383444,0.504708,0.436857,0.353183,0.349206


In [36]:
# compute season variance stats for each team
stats2 = pd.DataFrame()
for season in teams['Season'].unique():
    for team in teams['TeamID'].unique():
        if len(teams[(teams['Season'] == season) & (teams['TeamID'] == team) & (teams['Seed'] > 0)]) > 0:
            season_data = my_data[my_data['Season'] == season]
            w_data = season_data[season_data['WTeamID'] == team]
            w_data.columns = ['Season', 'TeamID', 'OppTeamID', 'FGM', 'OppFGM', 'FGA', 'OppFGA', 'FGM3', 'OppFGM3', 'FGA3', 'OppFGA3', 'FTM', 'OppFTM', 'FTA', 'OppFTA', 'Ast', 'OppAst', 'TO', 'OppTO', 'OR', 'OppOR', 'DR', 'OppDR', 'AdjScoreDiffPerPoss', 'OE', 'DE', 'OppOE', 'OppDE', 'PredictedAdjScoreDiffPerPoss']
            l_data = season_data[season_data['LTeamID'] == team]
            l_data = l_data[['Season', 'LTeamID', 'WTeamID', 'LFGM', 'WFGM', 'LFGA', 'WFGA', 'LFGM3', 'WFGM3', 'LFGA3', 'WFGA3', 'LFTM', 'WFTM', 'LFTA', 'WFTA', 'LAst', 'WAst', 'LTO', 'WTO', 'LOR', 'WOR', 'LDR', 'WDR', 'AdjScoreDiffPerPoss', 'LOE', 'LDE', 'WOE', 'WDE', 'PredictedAdjScoreDiffPerPoss']]
            l_data['AdjScoreDiffPerPoss'] = -1 * l_data['AdjScoreDiffPerPoss']
            l_data['PredictedAdjScoreDiffPerPoss'] = -1 * l_data['PredictedAdjScoreDiffPerPoss']
            l_data.columns = w_data.columns
            team_data = pd.concat([w_data, l_data])
            team_data['3ptRate'] = team_data['FGA3'] / team_data['FGA']
            team_data['Opp3ptRate'] = team_data['OppFGA3'] / team_data['OppFGA']
            team_data['Ast%'] = team_data['Ast'] / team_data['FGM']
            team_data['OppAst%'] = team_data['OppAst'] / team_data['OppFGM']
            team_data['eFG%'] = (team_data['FGM'] + .5 * team_data['FGM3']) / team_data['FGA']
            team_data['OppeFG%'] = (team_data['OppFGM'] + .5 * team_data['OppFGM3']) / team_data['OppFGA']
            team_data['3pt%'] = team_data['FGM3'] / team_data['FGA3']
            team_data['Opp3pt%'] = team_data['OppFGM3'] / team_data['OppFGA3']
            team_data['FT%'] = team_data['FTM'] / team_data['FTA']
            team_data['FTR'] = team_data['FTA'] / team_data['FGA']
            team_data['OppFTR'] = team_data['OppFTA'] / team_data['OppFGA']
            team_data['OR%'] = team_data['OR'] / (team_data['OR'] + team_data['OppDR'])
            team_data['OppOR%'] = team_data['OppOR'] / (team_data['OppOR'] + team_data['DR'])
            team_data['TO%'] = team_data['TO'] / (team_data['TO'] + team_data['FGA'] - team_data['OR'] + .44 * team_data['FTA'])
            team_data['OppTO%'] = team_data['OppTO'] / (team_data['OppTO'] + team_data['OppFGA'] - team_data['OppOR'] + .44 * team_data['OppFTA'])
            team_data['TotalPoss'] = team_data['TO'] + team_data['OppTO'] + team_data['FGA'] + team_data['OppFGA'] - team_data['OR'] - team_data['OR'] + .44 * (team_data['FTA'] + team_data['OppFTA'])
            team_data['GameScore'] = team_data['AdjScoreDiffPerPoss'] - team_data['PredictedAdjScoreDiffPerPoss']
            stats2 = pd.concat([stats2, pd.DataFrame({'Season': [season],
                                                     'TeamID': [team],
                                                     '3ptRateVar': [np.var(team_data['3ptRate'])],
                                                     'Opp3ptRateVar': [np.var(team_data['Opp3ptRate'])],
                                                     'eFG%Var': [np.var(team_data['eFG%'])],
                                                     'OppeFG%Var': [np.var(team_data['OppeFG%'])],
                                                     '3pt%Var': [np.var(team_data['3pt%'])],
                                                     'Opp3pt%Var': [np.var(team_data['Opp3pt%'])],
                                                     'Ast%Var': [np.var(team_data['Ast%'])],
                                                     'OppAst%Var': [np.var(team_data['OppAst%'])],
                                                     'FT%Var': [np.var(team_data['FT%'])],
                                                     'FTRVar': [np.var(team_data['FTR'])],
                                                     'OppFTRVar': [np.var(team_data['OppFTR'])],
                                                     'OR%Var': [np.var(team_data['OR%'])],
                                                     'OppOR%Var': [np.var(team_data['OppOR%'])],
                                                     'TO%Var': [np.var(team_data['TO%'])],
                                                     'OppTO%Var': [np.var(team_data['OppTO%'])],
                                                     'TotalPossVar': [np.var(team_data['TotalPoss'])],
                                                     'GameScoreVar': [np.var(team_data['GameScore'])]})])
stats2.head()

,Season,TeamID,3ptRateVar,Opp3ptRateVar,eFG%Var,OppeFG%Var,3pt%Var,Opp3pt%Var,Ast%Var,OppAst%Var,FT%Var,FTRVar,OppFTRVar,OR%Var,OppOR%Var,TO%Var,OppTO%Var,TotalPossVar,GameScoreVar
0,2008,1242.0,0.007283,0.006198,0.004911,0.004696,0.017420,0.009878,0.018027,0.011533,0.012707,0.024501,0.018192,0.007951,0.009547,0.002889,0.004971,148.488730,0.023759
0,2008,1314.0,0.003947,0.005160,0.005572,0.005751,0.011573,0.011725,0.015841,0.019245,0.009605,0.020250,0.013709,0.006448,0.007040,0.002062,0.003269,156.619013,0.014950
0,2008,1272.0,0.008861,0.007657,0.006284,0.006745,0.010864,0.013112,0.015508,0.018502,0.012012,0.030265,0.021677,0.006805,0.005892,0.002611,0.003380,158.480688,0.015790
0,2008,1181.0,0.005831,0.002779,0.005682,0.006119,0.009666,0.012727,0.007658,0.015275,0.010378,0.018600,0.021338,0.007295,0.007090,0.002577,0.003003,142.844273,0.013949
0,2008,1417.0,0.007557,0.010415,0.006688,0.006057,0.015887,0.011694,0.014693,0.015778,0.012248,0.015370,0.013996,0.008732,0.007553,0.002265,0.004128,136.532173,0.016354


In [37]:
stats_merge = pd.merge(stats, stats2, on = ['Season', 'TeamID'])
teams = pd.merge(teams, stats_merge, on = ['Season', 'TeamID'])
teams.head()

,TeamID,Season,TeamrankRating,TrankRating,OE,DE,Tempo,AvgTempo,WeightedRating,Seed,...,OppAst%Var,FT%Var,FTRVar,OppFTRVar,OR%Var,OppOR%Var,TO%Var,OppTO%Var,TotalPossVar,GameScoreVar
0,1242.0,2008,32.4,0.981585,120.970641,85.610492,69.5,67.317889,2.272611,1,...,0.011533,0.012707,0.024501,0.018192,0.007951,0.009547,0.002889,0.004971,148.488730,0.023759
1,1314.0,2008,29.4,0.957196,120.240748,91.770372,75.1,67.317889,2.070324,1,...,0.019245,0.009605,0.020250,0.013709,0.006448,0.007040,0.002062,0.003269,156.619013,0.014950
2,1272.0,2008,28.7,0.969683,113.254181,83.789458,70.7,67.317889,2.060585,1,...,0.018502,0.012012,0.030265,0.021677,0.006805,0.005892,0.002611,0.003380,158.480688,0.015790
3,1181.0,2008,28.5,0.960742,117.213494,88.761128,73.7,67.317889,2.032024,2,...,0.015275,0.010378,0.018600,0.021338,0.007295,0.007090,0.002577,0.003003,142.844273,0.013949
4,1417.0,2008,28.2,0.966422,116.350781,86.873950,66.2,67.317889,2.028528,1,...,0.015778,0.012248,0.015370,0.013996,0.008732,0.007553,0.002265,0.004128,136.532173,0.016354


In [38]:
for season in range(2008, 2020):
    print(str(season) + ": " + str(len(teams[teams['Season'] == season])))

2008: 65
2009: 65
2010: 65
2011: 68
2012: 68
2013: 68
2014: 68
2015: 68
2016: 68
2017: 68
2018: 68
2019: 68


### Tournament Matchups

In [57]:
ratings = teams[['Season', 'TeamID', 'WeightedRating']]
ratings.head()

,Season,TeamID,WeightedRating
0,2008,1242.0,2.272611
1,2008,1314.0,2.070324
2,2008,1272.0,2.060585
3,2008,1181.0,2.032024
4,2008,1417.0,2.028528


In [58]:
tournament_data = pd.read_csv('ncaam-march-mania-2021/MNCAATourneyDetailedResults.csv')
tournament_data.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [59]:
tournament_data['WScorePerPoss'] = 2 * tournament_data['WScore'] / (tournament_data['WFGA'] + 0.44 * tournament_data['WFTA'] + tournament_data['WTO'] - tournament_data['WOR'] + tournament_data['LFGA'] + 0.44 * tournament_data['LFTA'] + tournament_data['LTO'] - tournament_data['LOR'])
tournament_data['LScorePerPoss'] = 2 * tournament_data['LScore'] / (tournament_data['WFGA'] + 0.44 * tournament_data['WFTA'] + tournament_data['WTO'] - tournament_data['WOR'] + tournament_data['LFGA'] + 0.44 * tournament_data['LFTA'] + tournament_data['LTO'] - tournament_data['LOR'])

In [60]:
# The rating data is only from 2008 and 2010-2018 so we'll only use results from 2008 and 2010-2018
tournament_data = tournament_data.query('Season > 2007')[['Season', 'WTeamID', 'WScore', 'WScorePerPoss', 'LTeamID', 'LScore', 'LScorePerPoss']]

In [61]:
game_data = pd.merge(tournament_data, ratings, left_on = ['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])
game_data = pd.merge(game_data, ratings, left_on = ['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])
game_data = game_data.drop(columns = ['WTeamID', 'LTeamID']).rename(columns = {'WScore': 'Score_x', 
                                                                               'LScore': 'Score_y',
                                                                                'WScorePerPoss': 'ScorePerPoss_x',
                                                                                'LScorePerPoss': 'ScorePerPoss_y'})
game_data.head()

,Season,Score_x,ScorePerPoss_x,Score_y,ScorePerPoss_y,TeamID_x,WeightedRating_x,TeamID_y,WeightedRating_y
0,2008,69,1.039157,60,0.903614,1291.0,-0.216763,1164.0,-1.340130
1,2008,71,1.015446,70,1.001144,1181.0,2.032024,1125.0,0.137410
2,2008,85,1.330620,61,0.954915,1242.0,2.272611,1340.0,0.406410
3,2008,75,1.226292,56,0.915631,1242.0,2.272611,1424.0,1.195282
4,2008,72,1.101591,57,0.872093,1242.0,2.272611,1437.0,1.091378


We want to set up the data to have the "better" team as team x and the "worse" team as team y rather than x being the winner and y being the loser because when we make predictions we'll want to predict upsets instead of just picking whether an arbitrary team wins or loses. The code below rearranges team x and team y so team x always has the higher weighted rating. In the data above, the 9th row has team x with a lower weighted rating than team y.

In [62]:
# To make team x be the team with the higher rating and team y be the team with the lower rating
def switch_teams(row):
    # if rating x is less than rating y
    if row['WeightedRating_x'] < row['WeightedRating_y']:
        underdog = row['Score_x']  # "Worse" team's score
        favorite = row['Score_y'] # "Better" team's score
        row['Score_x'] = favorite
        row['Score_y'] = underdog
        underdog = row['ScorePerPoss_x']  # "Worse" team's score
        favorite = row['ScorePerPoss_y'] # "Better" team's score
        row['ScorePerPoss_x'] = favorite
        row['ScorePerPoss_y'] = underdog
        underdog = row['TeamID_x']  # "Worse" team's ID
        favorite = row['TeamID_y']  # "Better" team's ID
        row['TeamID_x'] = favorite
        row['TeamID_y'] = underdog
        underdog = row['WeightedRating_x']  # "Worse" team's rating
        favorite = row['WeightedRating_y']  # "Better" team's rating
        row['WeightedRating_x'] = favorite
        row['WeightedRating_y'] = underdog
    return row
                       
game_data = game_data.apply(switch_teams, axis = 1)
game_data.head(60)

,Season,Score_x,ScorePerPoss_x,Score_y,ScorePerPoss_y,TeamID_x,WeightedRating_x,TeamID_y,WeightedRating_y
0,2008.0,69.0,1.039157,60.0,0.903614,1291.0,-0.216763,1164.0,-1.340130
1,2008.0,71.0,1.015446,70.0,1.001144,1181.0,2.032024,1125.0,0.137410
2,2008.0,85.0,1.330620,61.0,0.954915,1242.0,2.272611,1340.0,0.406410
3,2008.0,75.0,1.226292,56.0,0.915631,1242.0,2.272611,1424.0,1.195282
4,2008.0,72.0,1.101591,57.0,0.872093,1242.0,2.272611,1437.0,1.091378
5,2008.0,59.0,0.948248,57.0,0.916104,1242.0,2.272611,1172.0,1.576862
6,2008.0,84.0,1.071702,66.0,0.842052,1242.0,2.272611,1314.0,2.070324
7,2008.0,75.0,1.056636,68.0,0.958016,1242.0,2.272611,1272.0,2.060585
8,2008.0,80.0,1.256676,67.0,1.052466,1243.0,1.603661,1425.0,1.505311
9,2008.0,74.0,1.284276,66.0,1.145436,1266.0,1.736178,1246.0,1.013676


In [63]:
game_data = pd.merge(game_data, teams, left_on = ['Season', 'TeamID_x'], right_on = ['Season', 'TeamID'])
game_data = pd.merge(game_data, teams, left_on = ['Season', 'TeamID_y'], right_on = ['Season', 'TeamID'])
game_data.head()

,Season,Score_x,ScorePerPoss_x,Score_y,ScorePerPoss_y,TeamID_x,WeightedRating_x,TeamID_y,WeightedRating_y,TeamID_x,...,OppAst%Var_y,FT%Var_y,FTRVar_y,OppFTRVar_y,OR%Var_y,OppOR%Var_y,TO%Var_y,OppTO%Var_y,TotalPossVar_y,GameScoreVar_y
0,2008.0,69.0,1.039157,60.0,0.903614,1291.0,-0.216763,1164.0,-1.340130,1291.0,...,0.029632,0.018188,0.028108,0.021425,0.005517,0.007589,0.002789,0.002284,268.621929,0.028297
1,2008.0,71.0,1.015446,70.0,1.001144,1181.0,2.032024,1125.0,0.137410,1181.0,...,0.015445,0.012537,0.016244,0.023928,0.006816,0.009025,0.003107,0.002055,199.356600,0.018555
2,2008.0,67.0,0.958786,73.0,1.044648,1181.0,2.032024,1452.0,1.675249,1181.0,...,0.018178,0.015968,0.021478,0.023766,0.009078,0.007205,0.002433,0.003884,152.203780,0.036482
3,2008.0,79.0,1.178050,75.0,1.118401,1462.0,1.714500,1452.0,1.675249,1462.0,...,0.018178,0.015968,0.021478,0.023766,0.009078,0.007205,0.002433,0.003884,152.203780,0.036482
4,2008.0,85.0,1.330620,61.0,0.954915,1242.0,2.272611,1340.0,0.406410,1242.0,...,0.014408,0.014316,0.026167,0.029929,0.008561,0.013031,0.002855,0.002063,117.979022,0.036102


In [64]:
game_data = game_data.loc[:,~game_data.columns.duplicated()]  # to remove duplicate columns
game_data.head()

,Season,Score_x,ScorePerPoss_x,Score_y,ScorePerPoss_y,TeamID_x,WeightedRating_x,TeamID_y,WeightedRating_y,TeamrankRating_x,...,OppAst%Var_y,FT%Var_y,FTRVar_y,OppFTRVar_y,OR%Var_y,OppOR%Var_y,TO%Var_y,OppTO%Var_y,TotalPossVar_y,GameScoreVar_y
0,2008.0,69.0,1.039157,60.0,0.903614,1291.0,-0.216763,1164.0,-1.340130,5.0,...,0.029632,0.018188,0.028108,0.021425,0.005517,0.007589,0.002789,0.002284,268.621929,0.028297
1,2008.0,71.0,1.015446,70.0,1.001144,1181.0,2.032024,1125.0,0.137410,28.5,...,0.015445,0.012537,0.016244,0.023928,0.006816,0.009025,0.003107,0.002055,199.356600,0.018555
2,2008.0,67.0,0.958786,73.0,1.044648,1181.0,2.032024,1452.0,1.675249,28.5,...,0.018178,0.015968,0.021478,0.023766,0.009078,0.007205,0.002433,0.003884,152.203780,0.036482
3,2008.0,79.0,1.178050,75.0,1.118401,1462.0,1.714500,1452.0,1.675249,23.6,...,0.018178,0.015968,0.021478,0.023766,0.009078,0.007205,0.002433,0.003884,152.203780,0.036482
4,2008.0,85.0,1.330620,61.0,0.954915,1242.0,2.272611,1340.0,0.406410,32.4,...,0.014408,0.014316,0.026167,0.029929,0.008561,0.013031,0.002855,0.002063,117.979022,0.036102


In [65]:
# Remove play-in games between 16 seeds
game_data = game_data.query('Seed_x != Seed_y or Seed_x != 16').reset_index(drop = True)

In [66]:
game_data.columns

Index(['Season', 'Score_x', 'ScorePerPoss_x', 'Score_y', 'ScorePerPoss_y',
       'TeamID_x', 'WeightedRating_x', 'TeamID_y', 'WeightedRating_y',
       'TeamrankRating_x', 'TrankRating_x', 'OE_x', 'DE_x', 'Tempo_x',
       'AvgTempo_x', 'Seed_x', '3ptRate_x', 'Ast%_x', 'FT%_x', 'OppFT%_x',
       'Opp3ptRate_x', 'OppAst%_x', 'EFG%_x', 'EFGD%_x', 'TOR%_x', 'TORD%_x',
       'ORB%_x', 'OppORB%_x', 'FTR_x', 'FTRD_x', '2P%_x', '2P%D_x', '3P%_x',
       '3P%D_x', '3ptRateVar_x', 'Opp3ptRateVar_x', 'eFG%Var_x',
       'OppeFG%Var_x', '3pt%Var_x', 'Opp3pt%Var_x', 'Ast%Var_x',
       'OppAst%Var_x', 'FT%Var_x', 'FTRVar_x', 'OppFTRVar_x', 'OR%Var_x',
       'OppOR%Var_x', 'TO%Var_x', 'OppTO%Var_x', 'TotalPossVar_x',
       'GameScoreVar_x', 'TeamrankRating_y', 'TrankRating_y', 'OE_y', 'DE_y',
       'Tempo_y', 'AvgTempo_y', 'Seed_y', '3ptRate_y', 'Ast%_y', 'FT%_y',
       'OppFT%_y', 'Opp3ptRate_y', 'OppAst%_y', 'EFG%_y', 'EFGD%_y', 'TOR%_y',
       'TORD%_y', 'ORB%_y', 'OppORB%_y', 'FTR_y', '

In [67]:
# Start with the stats for each team
matchups = game_data.drop(columns = ['Score_x', 'Score_y', 'ScorePerPoss_x', 'ScorePerPoss_y', 'TeamID_x', 'TeamID_y', 'AvgTempo_x', 'AvgTempo_y', 'WeightedRating_x', 'WeightedRating_y'])

# Response variable for upset probabilities
matchups['Upset'] = game_data['Score_x'] < game_data['Score_y']  
matchups['Upset'] = matchups['Upset'].astype('int64')

# Response variable for predicting spread
matchups['ScorePerPossDiff'] = game_data['ScorePerPoss_x'] - game_data['ScorePerPoss_y']

# Predictors

# Difference in NCAA tournament Seeds
matchups['SeedDiff'] = matchups['Seed_x'] - matchups['Seed_y']

# Trank Predicted Spread
matchups['TrankPredictedSpreadPerPoss'] = (game_data['OE_x'] +  game_data['DE_y'] - game_data['OE_y'] - game_data['DE_x']) / 100
matchups['TrankPredictedPoss'] = game_data['Tempo_x'] * game_data['Tempo_y'] / game_data['AvgTempo_x']
matchups['TrankPredictedSpread'] = matchups['TrankPredictedPoss'] * matchups['TrankPredictedSpreadPerPoss']

# Teamrank Predicted Spread
matchups['TeamrankPredictedSpread'] = game_data['TeamrankRating_x'] - game_data['TeamrankRating_y']
matchups['TeamrankPredictedSpreadPerPoss'] = matchups['TeamrankPredictedSpread'] / matchups['TrankPredictedPoss']

# tempo difference and absolute value of tempo difference for Trank tempo
matchups['TrankTempoDiff'] = game_data['Tempo_x'] - game_data['Tempo_y']
matchups['AbsTrankTempoDiff'] = abs(matchups['TrankTempoDiff'])

# Offensive vs defensive EFG% averages and differences
matchups['xOffyDefEFGAvg'] = (game_data['EFG%_x'] + game_data['EFGD%_y']) / 2
matchups['yOffxDefEFGAvg'] = (game_data['EFG%_y'] + game_data['EFGD%_x']) / 2
matchups['xOffyOffEFGDiff'] = matchups['xOffyDefEFGAvg'] - matchups['yOffxDefEFGAvg']

# Offensive vs defensive turnover rate averages and differences
matchups['xOffyDefTOAvg'] = (game_data['TOR%_x'] + game_data['TORD%_y']) / 2
matchups['yOffxDefTOAvg'] = (game_data['TOR%_y'] + game_data['TORD%_x']) / 2
matchups['xOffyOffTODiff'] = matchups['xOffyDefTOAvg'] - matchups['yOffxDefTOAvg']

# Offensive vs defensive rebound rate averages and differences
matchups['xOffRebAvg'] = (game_data['ORB%_x'] + game_data['OppORB%_y']) / 2
matchups['yOffRebAvg'] = (game_data['ORB%_y'] + game_data['OppORB%_x']) / 2
matchups['xOffyOffRebDiff'] = matchups['xOffRebAvg'] - matchups['yOffRebAvg']

# Offensive vs defensive FT rate averages and differences
matchups['xOffyDefFTRateAvg'] = (game_data['FTR_x'] + game_data['FTRD_y']) / 2
matchups['yOffxDefFTRateAvg'] = (game_data['FTR_y'] + game_data['FTRD_x']) / 2
matchups['xOffyOffFTRateDiff'] = matchups['xOffyDefFTRateAvg'] - matchups['yOffxDefFTRateAvg']

# Offensive vs defensive assist rate averages and differences
matchups['AbsxOffyDefAstDiff'] = abs(game_data['Ast%_x'] - game_data['OppAst%_y'])
matchups['AbsyOffxDefAstDiff'] = abs(game_data['Ast%_y'] - game_data['OppAst%_x'])
matchups['xOffyDefAstAvg'] = (game_data['Ast%_x'] + game_data['OppAst%_y']) / 2
matchups['yOffxDefAstAvg'] = (game_data['Ast%_y'] + game_data['OppAst%_x']) / 2
matchups['xOffyOffAstDiff'] = matchups['xOffyDefAstAvg'] - matchups['yOffxDefAstAvg']

# Sum of the variance in game possession of both teams
matchups['TotalPossVarSum'] = game_data['TotalPossVar_x'] + game_data['TotalPossVar_y']

# Sum of the variance in game performance of both teams (weighted by ratio of tempo and predicted tempo, ie sample sizes)
matchups['GameScoreVarSum'] = (game_data['Tempo_x'] / matchups['TrankPredictedPoss']) * game_data['GameScoreVar_x'] + (game_data['Tempo_y'] / matchups['TrankPredictedPoss']) * game_data['GameScoreVar_y']

# Naive upset probability using predicted spread and sum of variance
matchups['TrankNaiveUpsetProbability'] = norm.cdf(0, loc = matchups['TrankPredictedSpreadPerPoss'], scale = (0.5 * matchups['GameScoreVarSum']) ** 0.5)
matchups['TeamrankNaiveUpsetProbability'] = norm.cdf(0, loc = matchups['TeamrankPredictedSpreadPerPoss'], scale = (0.5 * matchups['GameScoreVarSum']) ** 0.5)

In [68]:
# Remove 1 v 16 matchups and 2 v 15 matchups, they were less predictive due to 15 and 16 seeds being much worse
matchups = matchups.query('SeedDiff > -13').reset_index(drop = True)
matchups.head()

,Season,TeamrankRating_x,TrankRating_x,OE_x,DE_x,Tempo_x,Seed_x,3ptRate_x,Ast%_x,FT%_x,...,xOffyDefFTRateAvg,yOffxDefFTRateAvg,AbsxOffyDefAstDiff,AbsyOffxDefAstDiff,xOffyDefAstAvg,yOffxDefAstAvg,TotalPossVarSum,GameScoreVarSum,TrankNaiveUpsetProbability,TeamrankNaiveUpsetProbability
0,2008.0,28.5,0.960742,117.213494,88.761128,73.7,2,0.390996,0.522826,0.694618,...,0.380141,0.330101,0.011372,0.152258,0.528512,0.552607,342.200873,0.030100,0.011556,0.022079
1,2008.0,28.5,0.960742,117.213494,88.761128,73.7,2,0.390996,0.522826,0.694618,...,0.396759,0.345179,0.004257,0.109056,0.520698,0.531006,295.048054,0.047594,0.312558,0.323143
2,2008.0,23.6,0.927155,115.187217,92.329124,65.8,3,0.371802,0.610714,0.750341,...,0.403042,0.342810,0.092145,0.018604,0.564642,0.594836,269.327900,0.072884,0.459461,0.493502
3,2008.0,32.4,0.981585,120.970641,85.610492,69.5,1,0.291796,0.627572,0.707756,...,0.360666,0.341935,0.079363,0.078373,0.587891,0.574597,266.467752,0.058420,0.030979,0.039768
4,2008.0,32.4,0.981585,120.970641,85.610492,69.5,1,0.291796,0.627572,0.707756,...,0.350930,0.316380,0.080697,0.071639,0.587224,0.571230,309.871774,0.046553,0.084577,0.069552


In [69]:
matchups.to_csv('mydata/mens/matchups.csv', index = False)